# Optimizing a 2-D inverted gaussian finction with GA's

In [1]:
from numpy.random import randint
from numpy.random import rand
import math

In [2]:
# The objective function is a two-dimensional inverted Gaussian function, centred at (7, 9)
#The minimum value of this function is at 7,9
def objective(x):
  y = math.exp(((x[0]-7)**2) + (x[1]-9)**2)
  return y

In [3]:
#we guesss the initial numbers in terms of bit-strings
#this function helps us decode the binary bit strings to real numbers
def decode(bounds, n_bits, bitstring):
	"""
	This function decodes binary bitstrings to numbers for each input and scales the value to fall within defined bounds.

	Parameters:
		bounds (list): A list of tuples representing the lower and upper bounds for each value to be decoded.
		n_bits (int): The number of bits used to represent each value.
		bitstring (list): A binary bitstring to be decoded.

	Returns:
		decoded (list): A list of decoded values.
	"""
	decoded = []  # Create an empty list to hold the decoded values
	largest = 2**n_bits  # The largest value that can be represented with the given number of bits
	for i in range(len(bounds)):
		# Extract the substring for the current value
		start, end = i * n_bits, (i * n_bits) + n_bits  # Define the start and end indices of the substring
		substring = bitstring[start:end]  # Extract the substring
		# Convert the substring to a string of characters
		chars = ''.join([str(s) for s in substring])  # Join the values in the substring together into a string of characters
		# Convert the string of characters to an integer
		integer = int(chars, 2)  # Convert the binary number string into an integer
		# Scale the integer to the desired range
		value = bounds[i][0] + (integer/largest) * (bounds[i][1] - bounds[i][0])  # Scale the integer to a value within the defined bounds
		# Store the decoded value
		decoded.append(value)
	return decoded

In [4]:
#selection function which helps us select the elites among the population
def selection(pop, scores, k=3):
    """
    Select the best individuals for the next generation based on their fitness (scores).
    This function randomly selects k individuals from the population and performs a tournament
    among them to choose the one with the best score.

    Parameters:
    pop (list): The population of individuals.
    scores (list): The fitness scores for each individual in the population.
    k (int, optional): The number of individuals to select from the population for the tournament.
                        Defaults to 3.

    Returns:
    individual: The best individual from the tournament.
    """
    # Randomly select one index from the population as the initial selection
    selection_ix = randint(len(pop))
    # Perform a tournament among k randomly selected individuals
    for ix in randint(0, len(pop), k-1):
        # Check if the current individual has a better score than the selected one
        if scores[ix] < scores[selection_ix]:
            # Update the selected individual if a better one is found
            selection_ix = ix
    # Return the best individual from the tournament
    return pop[selection_ix]

In [5]:
#cross-over function - to create offsprings of parents
def crossover(p1, p2, r_cross):
    """
    Create two children from two parents using the crossover operation.
    The children are created by copying the parents, and recombination occurs
    if a random value is less than the crossover rate.

    Parameters:
    p1 (list): The first parent.
    p2 (list): The second parent.
    r_cross (float): The crossover rate.

    Returns:
    list: A list containing the two children.
    """
    # Children are copies of the parents by default
    c1, c2 = p1.copy(), p2.copy()
    # Check if recombination should occur
    if rand() < r_cross:
        # Select a crossover point (not at the end of the string)
        pt = randint(1, len(p1)-2)
        # Perform crossover in the children
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    # Return the two children
    return [c1, c2]


In [6]:
#The crossover process can generate offsprings that are very similar to the parents. This might cause a new generation with low diversity.
# The mutation process solves this problem by changing the value of some features in the offspring at random.
import random

def mutation(bitstring, r_mut):
    """
    The mutation process changes the value of some features in the offspring at random to maintain the diversity in the population.
    A standard value for the mutation rate is 1/m where m is the number of features.

    Parameters:
    bitstring (list): A list of binary values representing the offspring
    r_mut (float): The mutation rate, typically 1/m where m is the number of features

    Returns:
    list: The modified bitstring after mutation

    """
    rand = random.random
    for i in range(len(bitstring)):
        # Check for a mutation
        if rand() < r_mut:
            # Flip the bit
            bitstring[i] = 1 - bitstring[i]
    return bitstring


In [7]:
# genetic algorithm implementation
def genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut):
    """
    The genetic algorithm that finds the optimal solution by performing the fitness assignment, selection, recombination, and mutation process repeatedly.
    Each iteration, the solution is more adapted to the environment.

    Parameters
    ----------
    objective: function
        The objective function that needs to be optimized.
    bounds: list of tuples
        The bounds of the solution.
    n_bits: int
        The number of bits used to encode the solution.
    n_iter: int
        The number of iterations to perform.
    n_pop: int
        The size of the population.
    r_cross: float
        The crossover rate.
    r_mut: float
        The mutation rate.

    Returns
    -------
    list
        The best solution and its evaluation.
    """
    # initialize the population with random bitstrings
    pop = [randint(0, 2, n_bits * len(bounds)).tolist() for _ in range(n_pop)]

    # track the best solution found so far
    best, best_eval = 0, objective(decode(bounds, n_bits, pop[0]))

    # iterate over generations
    for gen in range(n_iter):
        # decode the population
        decoded = [decode(bounds, n_bits, p) for p in pop]
        # evaluate all candidates in the population
        scores = [objective(d) for d in decoded]
        # check for a new best solution
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
                print(">%d, new best f(%s) = %f" % (gen,  decoded[i], scores[i]))

        # select parents
        selected = [selection(pop, scores) for _ in range(n_pop)]

        # create the next generation - children
        children = list()
        for i in range(0, n_pop, 2):
            # get selected parents in pairs
            p1, p2 = selected[i], selected[i + 1]
            # crossover and mutation
            for c in crossover(p1, p2, r_cross):
                # mutation
                mutation(c, r_mut)
                # store for next generation
                children.append(c)
        # replace the population
        pop = children
    return [best, best_eval]




In [8]:
# define range for input
bounds = [[-10.0, 10.0], [-10.0, 10.0]]
# define the total iterations
n_iter = 100
# bits per variable
n_bits = 16
# define the population size
n_pop = 100
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0 / (float(n_bits) * len(bounds))
# perform the genetic algorithm search
best, score = genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut)
print('###########################################################')
decoded = decode(bounds, n_bits, best)
print('The result is (%s) with a score of %f' % (decoded, score))

>0, new best f([5.19866943359375, 3.02825927734375]) = 78862251070346912.000000
>0, new best f([6.8499755859375, 4.62646484375]) = 207432612.897495
>0, new best f([4.4757080078125, 9.1595458984375]) = 600.345279
>0, new best f([9.11773681640625, 9.9462890625]) = 217.081360
>0, new best f([5.867919921875, 7.607421875]) = 25.050133
>1, new best f([5.885009765625, 8.02825927734375]) = 8.912754
>1, new best f([5.867919921875, 8.1011962890625]) = 8.080497
>1, new best f([6.97845458984375, 8.92730712890625]) = 1.005765
>2, new best f([6.9598388671875, 9.0570068359375]) = 1.004875
>3, new best f([6.9598388671875, 9.0386962890625]) = 1.003115
>4, new best f([6.97845458984375, 8.97308349609375]) = 1.001189
>7, new best f([6.97845458984375, 9.02313232421875]) = 1.001000
>7, new best f([6.9903564453125, 9.02923583984375]) = 1.000948
>9, new best f([6.99920654296875, 9.005126953125]) = 1.000027
>10, new best f([7.00164794921875, 9.00146484375]) = 1.000005
>15, new best f([7.00164794921875, 8.99932